## 基于技术指标的日内量化j交易策略

In [ ]:

# 基于技术指标的日内量化j交易策略
# 初始化函数
def init(context):
    #设置Boll模型参数
    context.timeperiod = 20 
    context.nbdevup = 2 
    context.nbdevdn = 2    
    #设置CCI模型参数
    context.ccitime = 60
    #设置双均线模型参数
    context.mashort = 5
    context.malong = 20
    #设置RSI模型参数
    context.rsi = 5
    #设置MACD模型参数
    context.MACDShort = 13 # 短周期平滑均线参数
    context.MACDLong = 33 # 长周期平滑均线参数
    context.MACDM = 13      # DIFF的平滑均线参数
    # 设置操作标的：
    # context.stockpool = ['002545.SZ','002941.SZ','600975.SH','600153.SH','600358.SH','000893.SZ','600985.SH','600716.SH','002567.SZ']
    context.stockpool = ['002625.SZ', '600657.SH', '300803.SZ', '002060.SZ', '688208.SH', '600359.SH']
    # 运作周期
    context.runstep_4 = [5]
    # 固定盈利
    context.win = 5 #盈利几个点卖出
    # 单笔买入仓位
    context.buypct = 0.05
    # 记录日内买入委托
    context.stocktrade = {}
    for stock in context.stockpool:
        context.stocktrade[stock] = []
    # 设置初识持仓
    set_holding_stocks(dict(zip(context.stockpool,[300]*len(context.stockpool))))
# 盘中运行函数
def handle_bar(context, bar_dict):
    # 获取当前分钟
    time = get_datetime().strftime('%M')
    # 循环股票
    for stock in context.stockpool:
        #判断盈利条件
        if len(context.stocktrade[stock])>0:
            for p in context.stocktrade[stock]:
                # 获取数据
                df = history(stock, ['close'], context.timeperiod*5, '1m', True, 'pre', is_panel=1)
                new_price = df['close'].iloc[-1]
                if new_price/p -1 > context.win/100:
                    # 卖出5%
                    order_percent(stock,-context.buypct)
                    print("卖出股票:{},卖出价格:{},盈利{}%".format(stock,new_price,context.win)) 
                    # 移除
                    context.stocktrade[stock].remove(p)
        # 循环周期判断
        for step in context.runstep_4:
            if int(time)%step==0:
                # 获取数据
                df = history(stock, ['close','high','low'], context.timeperiod*10, str(step)+'m', True, 'pre', is_panel=1)
                # 提取数据
                close_price = df['close']
                high_price = df['high']
                low_price = df['low']
                new_price = df['close'].iloc[-1]
                # 导入Python-talib库
                import talib
                # 计算指标，获取BOLL值
                upperband, middleband, lowerband = talib.BBANDS(close_price.values, timeperiod=context.timeperiod, nbdevup=context.nbdevup, nbdevdn=context.nbdevdn)
                # 计算指标，获取MACD值
                diff, dea, macd = talib.MACD(close_price.values,fastperiod = context.MACDShort,slowperiod = context.MACDLong,signalperiod = context.MACDM)
                # 计算指标，获取MA值
                mashort = talib.MA(close_price.values, timeperiod=context.mashort)
                malong = talib.MA(close_price.values, timeperiod=context.malong)
                # 计算指标，获取RSI值
                rsi = talib.RSI(close_price.values, timeperiod=context.rsi)
                # 计算指标，获取CCI值
                cci = talib.CCI(high_price.values, low_price.values, close_price.values, timeperiod=context.ccitime)
                # 判断买入条件
                if new_price< lowerband[-1] and cci[-1]<-100 and upperband[-1]<upperband[-2]:
                    # 买入5%
                    order_percent(stock,context.buypct)
                    # 记录这次买
                    context.stocktrade[stock].append(new_price)
                    print("买入条件1:{}分钟周期,买入{},买入价格{}".format(step,stock,new_price)) 
                # 判断买入条件2
                if rsi[-2]<20 and rsi[-1]>20 and rsi[-1]>rsi[-2]:
                    # 买入5%
                    order_percent(stock,context.buypct)
                    # 记录这次买
                    context.stocktrade[stock].append(new_price)
                    print("买入条件2:{}分钟周期,买入{},买入价格{}".format(step,stock,new_price)) 
                    
                    
                    